In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR


In [3]:

# Step 1: Set up the device
device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')

# Step 2: Load the pretrained FaceNet model
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Step 3: Freeze the pretrained layers (optional)
for param in facenet.parameters():
    param.requires_grad = False



In [ ]:
# Step 4: Add a new classification head
class FaceClassifier(nn.Module):
    def __init__(self, facenet, num_classes):
        super(FaceClassifier, self).__init__()
        self.facenet = facenet
        self.fc = nn.Linear(512, num_classes)  # Output dimension matches the number of classes

    def forward(self, x):
        embeddings = self.facenet(x)  # Extract embeddings
        out = self.fc(embeddings)     # Pass embeddings to classification head
        return out

# Example: Suppose we have 5 classes
num_classes = 5
model = FaceClassifier(facenet, num_classes).to(device)

# Step 5: Set up the training components
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Optimize only the classification head

# Step 6: Define a learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by 0.1 every 5 epochs

# Step 7: Prepare the data
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to match pretrained model expectations
])

# Load dataset (replace with your dataset path)
train_dataset = datasets.ImageFolder(root='../datasets/final_dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='../datasets/final_dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 8: Training Loop with tqdm and Learning Rate Scheduler
def train_model(model, train_loader, criterion, optimizer, num_epochs=10, scheduler=None):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Use tqdm to wrap the data loader and show progress
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()  # Reset the gradients
            loss.backward()  # Compute gradients
            optimizer.step()  # Update weights

            running_loss += loss.item()  # Track the loss

            # Update progress bar with the current loss
            progress_bar.set_postfix(loss=loss.item())

        # Print epoch loss after the loop
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

        # Step the scheduler if it is provided
        if scheduler:
            scheduler.step()

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10, scheduler=scheduler)



Epoch 1/10: 100%|██████████| 596/596 [03:50<00:00,  2.58batch/s, loss=0]      


Epoch 1/10, Loss: 0.0598


Epoch 2/10: 100%|██████████| 596/596 [03:45<00:00,  2.64batch/s, loss=0]      


Epoch 2/10, Loss: 0.0383


Epoch 3/10: 100%|██████████| 596/596 [04:08<00:00,  2.39batch/s, loss=0]       


...

Epoch 5/10: 100%|██████████| 596/596 [03:50<00:00,  2.58batch/s, loss=0]       


Epoch 5/10, Loss: 0.0242


Epoch 6/10: 100%|██████████| 596/596 [04:07<00:00,  2.40batch/s, loss=0]       


Epoch 6/10, Loss: 0.0235


Epoch 7/10:  59%|█████▉    | 352/596 [03:28<03:51,  1.05batch/s, loss=0.0184]  

In [ ]:
# Step 9: Evaluate the Model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# Evaluate the model
evaluate_model(model, test_loader)



In [ ]:
# Save the trained model's state dictionary
model_save_path = "../models/inception_vggface2_imfdb.pth"  # Path to save the model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")
